In [1]:
# import libraries
import pandas as pd

# change settings to see all columns
pd.set_option('display.max_columns', None)

# Clean species collection dataset

### Read file

In [2]:
species = pd.read_csv('species.csv')

### Five categories of species

We are categorizing species into five types.  
Not all species listed below may be found in the CSV file to be read.  
  
The species types are from table 5 of 464_longterm_trends.pdf

In [3]:
# species type categories
oceanic = ['Aristostomias scintillans', 'Bathylagus ochotensis', 'Bathylagus pacificus', 'Bathylagus wesethi', 'Ceratoscopelus townsendi', 'Chauliodus macouni', 'Chiasmodon niger', 'Cyclothone spp.', 'Diaphus theta', 'Diogenichthys atlanticus', 'Hygophum reinhardtii', 'Idiacanthus antrostomus', 'Melamphaes spp.', 'Microstoma spp.', 'Myctophidae', 'Myctophum nitidulum', 'Nannobrachium spp.', 'Nansenia candida', 'Notolychnus valdiviae', 'Notoscopelus resplendens', 'Paralepididae', 'Poromitra spp.', 'Protomyctophum crockeri', 'Scopelarchidae', 'Scopelogadus bispinosus', 'Scopelosaurus spp.', 'Stenobrachius leucopsarus', 'Sternoptychidae', 'Stomias atriventer', 'Symbolophorus californiensis', 'Tactostoma macropus', 'Tarletonbeania crenularis', 'Triphoturus mexicanus', 'Vinciguerria lucetia']
coastal_oceanic_fished = ['Engraulis mordax', 'Merluccius productus', 'Sardinops sagax', 'Scomber japonicus', 'Trachurus symmetricus', 'Cololabis saira', 'Icichthys lockingtoni', 'Leuroglossus stilbius', 'Tetragonurus cuvieri', 'Trachipterus altivelis']
coastal_fished = ['Microstomus pacificus', 'Paralabrax spp.','Paralichthys californicus', 'Parophrys vetulus', 'Sciaenidae', 'Scorpaenichthys marmoratus', 'Sebastes aurora', 'Sebastes paucispinis', 'Sebastes spp.', 'Sebastolobus spp.', 'Sphyraena argentea']
coastal_bycatch = ['Agonidae', 'Chromis punctipinnis', 'Hippoglossina stomata', 'Lyopsetta exilis', 'Pleuronichthys verticalis',  'Sebastes jordani', 'Symphurus atricaudus', 'Zaniolepisspp.']
coastal_unfished = ['Argentina sialis', 'Hypsoblennius spp.', 'Ophidion scrippsae', 'Oxylebius pictus']

species_types = [oceanic, coastal_oceanic_fished, coastal_fished, coastal_bycatch, coastal_unfished]
species_types_str = ['oceanic', 'coastal_oceanic_fished', 'coastal_fished', 'coastal_bycatch', 'coastal_unfished']

### Dictionary of categories

Create a dictionary of species:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;key = name of the species type  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;value = a list of all species corresponding to key.  
  
Only species found in the CSV file are added to the dictionary.

In [4]:
existing_types = [] # append with lists which correspond spcies that are found in the table

for each_type in species_types:
    existing_species = []
    for each_species in each_type:
        col_name = each_species.replace(' ', '.') # change species name to fit the column style (ie Aristostomias scintillans => Aristostomias.scintillans)
        if col_name in species.columns: # check if species exists in table
            existing_species.append(col_name) 
    existing_types.append(existing_species)
    

# convert existing_types list to dictionary
species_types_dict = dict()

for species_type, species_type_str in zip(existing_types, species_types_str):
    species_types_dict[species_type_str] = species_type

### Drop all columns that do not fit into a species type

If a speices in the CSV file is not found in any one of the five categories above, then remove the species column from the CSV file.  
Keep all non-species columns necessary for future data cleaning.

In [5]:
# non-species columns to keep
species_cols = ['S_L', 'S_S', 'year', 'season']

# append species_cols with species columns to keep
for species_type in species_types_dict.values():
    for species_name in species_type:
        species_cols.append(species_name)
        
# drop all columns not in the list species_cols
species = species[species_cols]

### Sum up populations of all species, for each species type

Create a column for each species type.  
Record the total count of all species that fall into the species type.

In [6]:
for species_type_name, species_lst in species_types_dict.items():
    species[species_type_name] = species.loc[:, species_lst].sum(axis=1)
    species.drop(columns=species_lst, inplace=True)

### Create a column of station ID

Station IDs show at what station (longitude, latitude) each row was recorded.

#### ⚠️ Some stations have multiple longitude/latitude values. We are using the most recent longitude/latitude values recorded for every station.

In [7]:
# change season to quarter
def season_to_quarter(season):
    if season == 'winter':
        return '1'
    if season == 'spring':
        return '2'
    if season == 'summer':
        return '3'
    if season == 'fall':
        return '4'
    
species['quarter'] = species['season'].apply(season_to_quarter)
species.drop(columns=['season'], inplace=True)

# change year/season to quarter
species['year'] = species['year'].astype(str)
species['quarter'] = species['year'] + ' ' + species['quarter']
species.drop(columns=['year'], inplace=True)

# create Sta_ID_x column to merge datasets on
species['S_L'] = species['S_L'].astype(str)
species['S_S'] = species['S_S'].astype(str)

species['S_L'] = species['S_L'].apply(lambda x: x.zfill(5))
species['S_S'] = species['S_S'].apply(lambda x: x.zfill(5))

species['station'] = species['S_L'] + " " + species['S_S']
species.drop(columns=['S_L', 'S_S'], inplace=True)

# rearrange column orders
species = species[['quarter', 'station', 'oceanic', 'coastal_oceanic_fished', 'coastal_fished', 'coastal_bycatch', 'coastal_unfished']]

# Clean station dataset

In [9]:
# read files
bottle = pd.read_csv('bottle.csv', encoding='latin')
cast = pd.read_csv('cast.csv', encoding='latin')

# drop columns
cast_cols = ['Cast_ID', 'Lat_Dec', 'Lon_Dec', 'Quarter', 'Year']
bottle_cols = ['Sta_ID', 'Depth_ID', 'T_degC', 'Depthm']

cast = cast[cast_cols]
bottle = bottle[bottle_cols] 

/Users/taekunkim/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (47,61) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/taekunkim/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (40,41,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
# merge cast and bottle dataframes
bottle['Cast_ID'] = bottle['Depth_ID'].apply(lambda x: x[:-8])
station = bottle.merge(cast, on='Cast_ID')
station.drop(columns=['Depth_ID', 'Cast_ID'], inplace=True)

# create quarter column
station['Year'] = station['Year'].astype(str)
station['Quarter'] = station['Quarter'].astype(str)

station['quarter'] = station['Year'] + ' ' + station['Quarter']

station.drop(columns=['Year', 'Quarter'], inplace=True)

# change comlumn name
station.rename(columns={'T_degC': 'Temperature',
                        'Lat_Dec': 'Latitude',
                        'Lon_Dec': 'Longitude'}, inplace=True)

In [13]:
# take the mean temperature measured from <100m
station = station.sort_values(['quarter', 'Sta_ID', 'Depthm'], ascending=True)
station = station[station['Depthm']<=100]
station = station.groupby(['quarter', 'Sta_ID'], as_index=False).agg({'Longitude': 'last', 'Latitude': 'last','Temperature': ['mean', 'std'], 'Depthm': ['mean', 'std']})
station.columns = ['quarter', 'station','longitude','latitude','temperature_mean','temperature_std', 'depth_mean', 'depth_std']

# Merge species and station datasets

In [14]:
# merge rockfish and station dataframe
data = species.merge(station, how='left', on=['station', 'quarter'])

In [15]:
# split year and quarter
data['year'] = data['quarter'].apply(lambda x: int(x[:4]))
data['quarter'] = data['quarter'].apply(lambda x: x[5])

# rearrange column orders
data = data[['year', 'quarter', 'station', 'temperature_mean', 'temperature_std', 'depth_mean', 'depth_std', 'oceanic', 'coastal_oceanic_fished', 'coastal_fished', 'coastal_bycatch', 'coastal_unfished', 'longitude', 'latitude']]

# sort table
data.sort_values(['year', 'quarter', 'station'], ascending=True, inplace=True)
data.reset_index(drop=True, inplace=True)

# export file
data.to_csv('data.csv', index=False)
data.to_json('data.json')

In [27]:
temp = data[data['year']==1960]
# temp[temp['quarter']==3]

In [28]:
temp = temp[temp['quarter']=='1'].reset_index(drop=True)
temp = temp[temp['latitude'].notna()]
temp = temp[temp['longitude'].notna()]
temp.to_csv('1960.csv', index=False)